<p style="font-family: Arial; font-size:2.75em;color:blue; font-style:bold"><br>
Projet 6 :<br><br>
Catégorisez automatiquement des questions<br><br>
</p><br>

* **Data Source / Data Source :** 
    - Data :
        - Outils d'extraction de data : https://data.stackexchange.com/stackoverflow
        - Script : https://github.com/EricJacquesPro/TextCategorization/blob/master/SQL/ExtractionDonnees.sql
    - Source Python : https://github.com/EricJacquesPro/TextCategorization
* **Description / Description : ** Projet 6 de la formation Data Scientist Proposée par OpenClassroom
    - Analyse Supervised
    - Entrainement Supervised
* **Auteur / Author : ** Eric JACQUES 
* **Date : ** 16/06/2020 

<p style="font-family: Arial; font-size:1.75em;color:#2462C0; font-style:bold"><br>
I - Sélection des datas entrantes</p>

In [1]:
def scoring(x_test, y_true, clf, lb, mode_supervise_with_lda = False, lda_model = None):
    debut_scoring = time.time()
    nb_tag_1 = 0.0
    nb_tag_5 = 0.0
    classes = lb.classes_
    print(x_test.shape[0])
    no_top_words = 5
    for i in range(x_test.shape[0]):

        debut_boucle = time.time()
        #for i in range(x_test.shape[0]):
        text_projection = x_test
        if(mode_supervise_with_lda):
            text_projection = lda_model.transform(x_test[i])
        '''
        print (text_projection)
        print (text_projection.shape)
        '''
        #voir le temps
        #voir nb tag
        predicted = clf.predict_proba(text_projection)
        '''
        print (predicted)
        print (len(predicted))
        '''
        tempTag = [(1-item[0][0]) for item in predicted]
        list_id = [[i, x] for i, x in enumerate(tempTag) if x > 0.0050]
        '''
        tempTag = predicted[0]
        list_id = [[i, x] for i, x in enumerate(tempTag) ]#if x > 0.0050]
        '''
        '''
        print (predicted)
        print (predicted.shape)
        '''
        list_id_sorted = sorted(list_id, reverse=True, key=lambda x: x[1])
        #print (list_id_sorted)

        list_id_sorted_suggested = [x[0] for i, x in enumerate(list_id_sorted[:-no_top_words - 1:-1])]
        #print(list_id_sorted_suggested)
        prediction = [classes[id] for id in list_id_sorted_suggested]
        #print (str(prediction))


        l_y = [[i, x] for i, x in enumerate(y_true[i]) if x > 0]
        l_y_tagged = [x[0] for i, x in enumerate(l_y[:-no_top_words - 1:-1])]
        l_y_tags = [classes[id] for id in l_y_tagged]
        #print (l_y_tags)

        check_1 = False
        check_1 = any(item in prediction for item in l_y_tags)

        if check_1 is True:
            nb_tag_1 = nb_tag_1 + 1
        """    print("The list {} contains some elements of the list {}".format(prediction, l_y_tags))    
        else :
            print("No, List1 doesn't have any elements of the List2.")
        """

        check_5 = False
        check_5 = all(item in prediction for item in l_y_tags)
        if check_5 is True:
            nb_tag_5 = nb_tag_5 + 1
        """    print("The list {} contains all elements of the list {}".format(prediction, l_y_tags))    
        else :
            print("No, List1 doesn't have any elements of the List2.")
        """
        #str([tag for tag in y_true[i]]if tag ==1)
        fin_boucle = time.time()
        print("temps pour la boucle scorring : {0} s".format(fin_boucle - debut_boucle))
        del fin_boucle, debut_boucle
    fin_scoring = time.time()
    print("temps pour le function scoring : {0} s".format(fin_scoring - debut_scoring))
    del fin_scoring, debut_scoring
    return nb_tag_1, (100.0 * nb_tag_1 / float(x_test.shape[0])), nb_tag_5, (100.0 * nb_tag_5 / float(x_test.shape[0]))


In [2]:

import time

In [8]:
import sys
import os
import numpy as np
import pandas as pd
sys.path.insert(0, './Python/')
'''
python class for the data engineering (reading, cleaning, training...)
localised in ./Python
'''
from tagText import TagText 
tagText = TagText()

tagText.nombre_post_entree = 5000#500#00
tagText.precision = 5000#500#00

print(tagText.urlDirectory)#folder for cvs file generated by SQL query (cf. I) 
print(tagText.fileName)#name of the cvs file generated by SQL query (cf. I)
debut_lecture = time.time()
data_question = tagText.read_source()
print(data_question.head())

fin_lecture = time.time()
print("temps pour la lecture : {0} s".format(fin_lecture - debut_lecture))
del fin_lecture, debut_lecture

debut_preprocess = time.time()
data_preprocessed = data_question['body'].apply(tagText.preprocessing)
fin_preprocess = time.time()
print("temps pour le pre processing : {0} s".format(fin_preprocess - debut_preprocess))
del fin_preprocess, debut_preprocess

print(data_preprocessed.head())

Data/
QuestionVsTags.csv
                                                body  \
0  <p>Here is a piece of C++ code that shows some...   
1  <p>I accidentally committed the wrong files to...   
2  <p>I want to delete a branch both locally and ...   
3  <p>What are the differences between <code>git ...   
4  <p>What is the use of the <code>yield</code> k...   

                                               title  \
0  Why is processing a sorted array faster than p...   
1  How do I undo the most recent local commits in...   
2  How do I delete a Git branch locally and remot...   
3  What is the difference between 'git pull' and ...   
4                  What does the "yield" keyword do?   

                                                tags  \
0  <java><c++><performance><optimization><branch-...   
1  <git><version-control><git-commit><undo><pre-c...   
2  <git><version-control><git-branch><git-push><g...   
3        <git><version-control><git-pull><git-fetch>   
4    <python><iterato

In [6]:
def supervised_prepare_tagV2(self, data_preprocessed, data_tag):
    '''
    prepare classifier and class from file for supervised model
    '''
    from sklearn.pipeline import Pipeline
    from sklearn import metrics
    from sklearn.model_selection import train_test_split
    from sklearn.feature_extraction.text import TfidfVectorizer

    y_all = [
        item[:-1].split(',')#-1 car il y a un ',' à la fin de la ligne
        for item in data_tag
    ]

    #print(y_train_tag)
    lb = tagText.MultiLabelBinarizer()
    Y_all = lb.fit_transform(y_all)


    # 80/20 split
    X_train, X_test, y_train, y_test = train_test_split(
        data_preprocessed, y_all, test_size=0.1,train_size=0.9, random_state=0)
    y_train = lb.transform(y_train)
    y_test = lb.transform(y_test)

    classifier = Pipeline([
        ('vectorizer', self.CountVectorizer()),
        ('tfidf', self.TfidfTransformer()),
        (
            'clf',
            self.RandomForestClassifier(
                n_estimators=100,
                max_depth=2,
                random_state=0
            )
        )
    ])  
    classifier.fit(X_train, y_train)
    '''
    print (len(X_test))
    print (len(y_test))
    print (y_test[0])
    print (X_test)
    '''
    nb1, score1, nb5, score5 = scoring(X_test, y_test, classifier, lb, False, None)
    
    print("score 1 occurence {}".format(score1))
    print("score 5 occurences {}".format(score5))
    
    return classifier, lb.classes_


In [9]:
classifier, tags = supervised_prepare_tagV2(tagText, data_preprocessed, data_question['SelectedTags'])
#del classifier, tags

500
temps pour la boucle scorring : 3.340918779373169 s
temps pour la boucle scorring : 3.3400306701660156 s
temps pour la boucle scorring : 3.377286911010742 s
temps pour la boucle scorring : 3.278393507003784 s
temps pour la boucle scorring : 3.3893542289733887 s
temps pour la boucle scorring : 3.377225399017334 s
temps pour la boucle scorring : 3.4892265796661377 s
temps pour la boucle scorring : 3.349573850631714 s
temps pour la boucle scorring : 3.41337513923645 s
temps pour la boucle scorring : 3.400981903076172 s
temps pour la boucle scorring : 3.276733875274658 s
temps pour la boucle scorring : 3.402024507522583 s
temps pour la boucle scorring : 3.3889777660369873 s
temps pour la boucle scorring : 3.3197669982910156 s
temps pour la boucle scorring : 3.4324684143066406 s
temps pour la boucle scorring : 3.3171019554138184 s
temps pour la boucle scorring : 3.6332778930664062 s
temps pour la boucle scorring : 3.3929193019866943 s
temps pour la boucle scorring : 3.3693416118621826 s

temps pour la boucle scorring : 3.4637324810028076 s
temps pour la boucle scorring : 3.3969531059265137 s
temps pour la boucle scorring : 3.318671941757202 s
temps pour la boucle scorring : 3.441467523574829 s
temps pour la boucle scorring : 3.680150270462036 s
temps pour la boucle scorring : 3.6173243522644043 s
temps pour la boucle scorring : 4.340712547302246 s
temps pour la boucle scorring : 3.6033549308776855 s
temps pour la boucle scorring : 3.9254589080810547 s
temps pour la boucle scorring : 3.5415613651275635 s
temps pour la boucle scorring : 3.4193191528320312 s
temps pour la boucle scorring : 3.448430299758911 s
temps pour la boucle scorring : 3.8686530590057373 s
temps pour la boucle scorring : 4.2338714599609375 s
temps pour la boucle scorring : 4.955739498138428 s
temps pour la boucle scorring : 5.731761455535889 s
temps pour la boucle scorring : 4.56382155418396 s
temps pour la boucle scorring : 3.784930944442749 s
temps pour la boucle scorring : 3.4932801723480225 s
tem

temps pour la boucle scorring : 8.091261625289917 s
temps pour la boucle scorring : 8.19490933418274 s
temps pour la boucle scorring : 7.333496809005737 s
temps pour la boucle scorring : 7.3256995677948 s
temps pour la boucle scorring : 7.658655405044556 s
temps pour la boucle scorring : 8.091978311538696 s
temps pour la boucle scorring : 7.811148166656494 s
temps pour la boucle scorring : 8.133432149887085 s
temps pour la boucle scorring : 8.109160423278809 s
temps pour la boucle scorring : 8.437700510025024 s
temps pour la boucle scorring : 7.910480737686157 s
temps pour la boucle scorring : 7.569324970245361 s
temps pour la boucle scorring : 9.31679368019104 s
temps pour la boucle scorring : 9.165345668792725 s
temps pour la boucle scorring : 8.189829111099243 s
temps pour la boucle scorring : 7.8493921756744385 s
temps pour la boucle scorring : 8.1580069065094 s
temps pour la boucle scorring : 7.921298265457153 s
temps pour la boucle scorring : 8.462272644042969 s
temps pour la bou

temps pour la boucle scorring : 9.914443492889404 s
temps pour la boucle scorring : 10.16594910621643 s
temps pour la boucle scorring : 9.252251386642456 s
temps pour la boucle scorring : 9.48597002029419 s
temps pour la boucle scorring : 9.806456804275513 s
temps pour la boucle scorring : 10.04554533958435 s
temps pour la boucle scorring : 9.557885885238647 s
temps pour la boucle scorring : 9.44791293144226 s
temps pour la boucle scorring : 9.581208944320679 s
temps pour la boucle scorring : 9.058079957962036 s
temps pour la boucle scorring : 9.222900390625 s
temps pour la boucle scorring : 9.63104510307312 s
temps pour la boucle scorring : 8.942755222320557 s
temps pour la boucle scorring : 9.555473327636719 s
temps pour la boucle scorring : 9.535664081573486 s
temps pour la boucle scorring : 9.285393714904785 s
temps pour la boucle scorring : 9.72128176689148 s
temps pour la boucle scorring : 9.809279680252075 s
temps pour la boucle scorring : 10.054394245147705 s
temps pour la bouc

In [6]:
classifier, tags = tagText.supervised_prepare_tagV2(data_preprocessed, data_question['SelectedTags'])
del classifier, tags



100
100
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0]
90     difference function decorated staticmethod one...
254    python parse numeric string like 5452222 corre...
283    im trying open url new tab opposed popup windo...
445    please describe typescript language is javascr...
461    valid returns string 10 javascript more exampl...
15     would explain javascript closures someone know...
316    let prefix saying know foreach is use it quest...
489    ive using git recently downloaded update find ...
159    know do try  something may fail except  anythi...
1

In [6]:

message = 'git est un outils de code source. C\'est bien'
print(message)



print(tagText.supervised_predict(message, classifier, tags))

git est un outils de code source. C'est bien
['data-structures', 'foreach', 'initialization', 'powershell', 'parsing']


In [7]:
print(tagText.supervised_predict(data_preprocessed[0], classifier, tags ))

['layout', 'pandas', 'dataframe', 'data-structures', 'android-edittext']


In [8]:
print (tags)

['.net' 'ajax' 'algorithm' 'android' 'android-activity' 'android-edittext'
 'android-layout' 'angularjs' 'apache-flex' 'arraylist' 'arrays' 'asp.net'
 'assembly' 'asynchronous' 'attributes' 'authentication' 'autocomplete'
 'bash' 'browser' 'c' 'c#' 'c++' 'c++11' 'caching' 'casting' 'char'
 'character-encoding' 'checkbox' 'class' 'cocoa-touch' 'coldfusion'
 'collections' 'command-line' 'constructor' 'css' 'css-selectors' 'csv'
 'curl' 'data-binding' 'data-structures' 'database' 'dataframe' 'date'
 'datetime' 'dependency-injection' 'design-patterns' 'dictionary'
 'directory' 'docker' 'dom' 'drop-down-menu' 'duplicates' 'dynamic'
 'eclipse' 'ecmascript-6' 'emacs' 'email' 'enums' 'error-handling'
 'event-handling' 'events' 'excel' 'exception' 'exception-handling'
 'express' 'facebook' 'file' 'file-io' 'file-upload' 'flash'
 'floating-point' 'for-loop' 'foreach' 'formatting' 'forms' 'function'
 'functional-programming' 'garbage-collection' 'gcc' 'geometry' 'git'
 'github' 'grep' 'gulp' 'has

In [9]:
tagText.supervised_predict('c c# .Net', classifier, tags )

"['initialization', 'package', 'formatting', 'user-interface', 'database']"

In [10]:
tagText.supervised_predict('how to convert int in flaot in c c# .Net', classifier, tags, 120000 )

"['formatting', 'caching', 'layout', 'foreach', 'email', 'time', 'data-structures', 'powershell', 'parsing', 'android-edittext', 'objective-c', 'ios', 'pandas', 'dataframe', 'hashmap', 'numbers', 'replace', 'angularjs', 'listview', 'c++11', 'constructor', 'terminal', 'serialization', 'variables', 'ajax', 'dom', 'file', 'http-headers', 'split', 'indexing', 'arraylist', 'mysql', 'merge', 'html-table', 'postgresql', 'floating-point', 'grep', 'validation', 'unix', 'exception', 'rest', 'scope', 'type-conversion', 'command-line', 'npm', 'design-patterns', 'security', 'file-io', 'tsql', 'datetime', 'function', 'date', 'github', 'scripting', 'collections', 'regex', 'pointers', 'performance', 'stream', 'io', 'enums', 'object', 'vim', 'syntax', 'docker', 'null', 'algorithm', 'http', 'url', 'linux', 'sql-server', 'random', 'oop', 'directory', 'json', 'casting', 'php', 'c', 'node.js', 'dictionary', 'loops', 'shell', 'sql', 'android', 'version-control', '.net', 'list', 'c++', 'bash', 'arrays', 'css